# 2016 Summer Q8
# MPI (Dr. Huang)

## 1)

mpiCC summer16q8_1.cpp

mpirun -np 4 ./a.out

#include <iostream>
#include "mpi.h"

using namespace std;

int main()
{
	int i, j;
	int n = 4;
	int m = 3;
	double B[n][m];
	double a[m];
	
	//build matrix and vector
	for (i = 0; i < n; i++)
	{
		for (j = 0; j < m; j++)
		{
			B[i][j] = i + j;
			if (i == 0)
			{
				a[j] = 1;
			}
		}
	}

	int ierr, p, id;
	ierr = MPI_Init(NULL, NULL);
	ierr = MPI_Comm_size(MPI_COMM_WORLD, &p);
	ierr = MPI_Comm_rank(MPI_COMM_WORLD, &id);
	
	//each processor takes a row of the matrix B and computes
	//its dot product with the vector a
	double row_dot_product = 0;
	for (i = 0; i < m; i++)
	{
		row_dot_product += B[id][i] * a[i];
	}

	double matrix_vector_product = 0;
	MPI_Allreduce(&row_dot_product, &matrix_vector_product, 1, MPI_DOUBLE, MPI_SUM, MPI_COMM_WORLD);
	
	if (id == 0)
	{
		cout << matrix_vector_product << endl;
	}

	MPI_Finalize();
}

## 2)

mpiCC summer16q8_2.cpp

mpirun -np 4 ./a.out

#include <iostream>
#include "mpi.h"

using namespace std;

int main(){
	int i, j;
	int n = 4;
	int m = 3;
	double B[n][m];
	double a[m];
	
	//build matrix and vector
	for (i = 0; i < n; i++){
		for (j = 0; j < m; j++){
			B[i][j] = i + j;
			if (i == 0){
				a[j] = 1;
			}
		}
	}

	int ierr, p, id;
	ierr = MPI_Init(NULL, NULL);
	ierr = MPI_Comm_size(MPI_COMM_WORLD, &p);
	ierr = MPI_Comm_rank(MPI_COMM_WORLD, &id);
	
	//each processor takes a row of the matrix B and computes
	//its dot product with the vector a
	double row_dot_product = 0;
	for (i = 0; i < m; i++){
		row_dot_product += B[id][i] * a[i];
	}

	double matrix_vector_product = 0;
	if (id == 0){
		matrix_vector_product += row_dot_product;
	}
	for (i = 1; i < n; i++){
		if (id == i){
			MPI_Send(&row_dot_product, 1, MPI_DOUBLE, 0, 0, MPI_COMM_WORLD);
		}
		if (id == 0){
			MPI_Recv(&row_dot_product, 1, MPI_DOUBLE, i, 0, MPI_COMM_WORLD,
				MPI_STATUS_IGNORE);
			cout << "message received from: " << i << endl;
			matrix_vector_product += row_dot_product;
		}
	}

	if (id == 0){
		cout << matrix_vector_product << endl;
	}

	MPI_Finalize();
}

## 3)

mpiCC summer16q8_3.cpp

mpirun -np 2 ./a.out

#include <iostream>
#include "mpi.h"

using namespace std;

int main(){
	int i, j;
	int n = 4;
	int m = 3;
	double B[n][m];
	double a[m];

	//enter number of processors to be used
	//(this code assumes: r<n, and r is a factor of n
	int r = 2;
	int rows_per_id = n/r;
	
	//build matrix and vector
	for (i = 0; i < n; i++){
		for (j = 0; j < m; j++){
			B[i][j] = i + j;
			if (i == 0){
				a[j] = 1;
			}
		}
	}

	int ierr, p, id;
	ierr = MPI_Init(NULL, NULL);
	ierr = MPI_Comm_size(MPI_COMM_WORLD, &p);
	ierr = MPI_Comm_rank(MPI_COMM_WORLD, &id);
	
	//each processor takes a row of the matrix B and computes
	//its dot product with the vector a
	double row_dot_product = 0;
	for (i = 0; i < m; i++){
		for (j = 0; j < rows_per_id; j++){
			row_dot_product += B[id*rows_per_id+j][i] * a[i];
		}
	}

	double matrix_vector_product = 0;
	if (id == 0){
		matrix_vector_product += row_dot_product;
	}
	for (i = 1; i < r; i++){
		if (id == i){
			MPI_Send(&row_dot_product, 1, MPI_DOUBLE, 0, 0, MPI_COMM_WORLD);
		}
		if (id == 0){
			MPI_Recv(&row_dot_product, 1, MPI_DOUBLE, i, 0, MPI_COMM_WORLD,
				MPI_STATUS_IGNORE);
			cout << "message received from: " << i << endl;
			matrix_vector_product += row_dot_product;
		}
	}

	if (id == 0){
		cout << matrix_vector_product << endl;
	}

	MPI_Finalize();
}

## 4)

Dr. Burkardt:
"Each worker is given 
a copy of x, and then is fed one row of A.  As soon as it computes 
B(I) = A(I,1:N)\*x(1:N), it is given another column of A, unless
there are no more, in which case it is sent a "terminate" message. 
Thus, a faster process will be given more work to do."
https://people.sc.fsu.edu/~jburkardt/cpp_src/mpi/matvec_mpi.cpp